# SEA JCLplus+ JCL Defects Metrics Analysis Notebook

## Description

This notebook reads the JCLplus+ Global Record, performs a descriptive analysis of JCL defects and calculate global defects indices.

<h2>Table of Contents</h2>
<div class="alert alert-block alert-info" style="margin-top: 20px">
    <ul>
          <a href="#business">1. Business Understanding</a><br>
          <a href="#prep">2. Data Preparation</a>
              <ul>
                <li><a href="read">Data Acuisition - JCLplus+ Global Record</a></li>
                <li><a href="transform">Perform Transformations</a></li>
                <li><a href="info">Records Information</a></li>
              </ul>
          <a href="#business">3. Analytic Approach</a><br>
            <ul>
                <li><a href="desc">Descriptive Analysis</a></li>
                <li><a href="index">Indices</a></li>
                <li><a href="visual">Data Visualization</a></li>
                   <ul>
                    <li><a href="def">Total Defects by Error Type</a></li>
                    <li><a href="def_date">Total Defects by Date</a></li>
                   </ul>    
              </ul>
         
                
</div>

<hr>

## 1. Introduction: Business Problem

JCLplus+ is equipped with an engine that not only produces Summary defect reports but also generates two typles of records(comma-separated CSV format) with significant information that can be easily imported by any RDBMS, spreadsheet or other reporting system. 

The global statistics record offers information of the overall quality of the JCL change process and the problems captured by the rules defined in the product.      
                                     	
This set of metrics can help to determine a course of action to identify categories and priorities of repetitive problems that may impact the time to delivery of JCL components. These rates can also serve as a baseline for benchmarking purposes and validate the proper adoption of Best Practices procedures. These reports can also be broken down by failure type and by promotion stage within the SCM.

## 2. Data Preparation

## Importing Libraries

In [1]:
# Importing Pandas Library
import pandas as pd

#Importing other Libraries
import sys, os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline 

### _Read JCLplus+ Global File_
_Note: Change the path of the JClplus+ Global CSV file_

In [2]:
jpd=pd.read_csv('C:\\sea\\logstash-6.0.1\\data\\jpdetail.csv',header=None)
jpd.columns=['sdate', 'stime' , 'edate', 'etime', 'jpjob' , 'stepname' ,
             'smfid' , 'rel', 'userid' , 'mode' , 'rules', 'warn', 'error' , 'stmtserr', 'rcode' ,
             'ruleds' , 'rulename', 'rexxrule', 'exitrule' , 'msgrange', 'msgsuf' ,
             'msgsev' , 'msgprefix', 'msgnum', 'msgtype' , 'msgtxt' ,'pds' , 'member' , 'jobname', 'trail']
jpd.head()

,sdate,stime,edate,etime,jpjob,stepname,smfid,rel,userid,mode,...,msgsuf,msgsev,msgprefix,msgnum,msgtype,msgtxt,pds,member,jobname,trail
0,43321,09:26:26:00,8/9/2018,09:26:27:00,SEAUSER,ISPFPROC,JRCM,4.4G,SEAUSER,EDIT,...,J,4,XRP,2412,WARN,"DYNALLOC FAILED, UNABLE TO VERIFY MEMBER IN: J...",JRC1.SEAUSER.JCL.CNTL,#ACS#TR0,XI1XXXB,
1,43321,09:26:26:00,8/9/2018,09:26:27:00,SEAUSER,ISPFPROC,JRCM,4.4G,SEAUSER,EDIT,...,J,4,XRP,2350,WARN,DATASET NOT FOUND: JRC1.SEAUSER.JCL ...,JRC1.SEAUSER.JCL.CNTL,#ACS#TR0,XI1XXXB,
2,43321,09:26:26:00,8/9/2018,09:26:27:00,SEAUSER,ISPFPROC,JRCM,4.4G,SEAUSER,EDIT,...,J,4,XRP,2350,WARN,DATASET NOT FOUND: JRC1.SLBJES2A.PLUSPACK.LINK...,JRC1.SEAUSER.JCL.CNTL,#ACS#TR0,XI1XXXB,
3,43321,09:26:26:00,8/9/2018,09:26:27:00,SEAUSER,ISPFPROC,JRCM,4.4G,SEAUSER,EDIT,...,J,4,XRP,2307,ERROR,NO PARAMETERS ON STATEMENT ...,JRC1.SEAUSER.JCL.CNTL,#ACS#TR0,XI1XXXB,
4,43321,09:26:26:00,8/9/2018,09:26:27:00,SEAUSER,ISPFPROC,JRCM,4.4G,SEAUSER,EDIT,...,J,8,XRP,2114,ERROR,"MISSING ""="" SIGN AFTER ""DSN"" KEYWORD ...",JRC1.SEAUSER.JCL.CNTL,#ACS#TR0,XI1XXXB,


In [3]:
jpd.sdate=jpd.sdate.astype('datetime64[ns]')
jpd.edate=jpd.edate.astype('datetime64[ns]')

In [4]:
 # Import sqlalchemy Python Toolkit and convert dataframes to tables.
from sqlalchemy.types import Integer, Text, String, DateTime
from sqlalchemy import create_engine
engine = create_engine('sqlite://', echo=False)

jpd.to_sql("jpdetail",
            engine,
            if_exists='replace',
            index=False)

print('*** jpdetail Sqlalchemy SQL Tables Created')

*** jpdetail Sqlalchemy SQL Tables Created


In [5]:
# Find Top 10 Defects
jpd_top_d = pd.read_sql("SELECT msgnum, count(msgnum) as count  FROM jpdetail \
                          GROUP BY msgnum ORDER BY count DESC LIMIT 10" , 
                     con=engine, 
                     parse_dates=['date'])

jpd_top_d.reset_index(drop=True, inplace=True)

print('------------------------------------------------------')
print('Top 10 JCL Defect Type')
print('------------------------------------------------------')
print(jpd_top_d)


------------------------------------------------------
Top 10 JCL Defect Type
------------------------------------------------------
   msgnum  count
0     354   1212
1    2350    720
2    2328    687
3    2423    147
4    2307    139
5    2324    128
6    2405    114
7    2244     68
8    2357     51
9    2412     41
